# Examing and Selecting Data Sources:

In [1]:
import pandas as pd
from datetime import datetime
import os    
from dotenv  import load_dotenv
from pathlib import Path

## 1) Testing & Importing Data from Alpaca:

In [7]:
import alpaca_trade_api as tradeapi

In [10]:
load_dotenv(dotenv_path = Path("C:/Users/Leon/API_keys/.env"))

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
print(type(alpaca_secret_key))
print(type(alpaca_api_key))

# Creating Alpaca Object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

<class 'str'>
<class 'str'>


In [11]:
ticker_alpaca = "BTC"
timeframe = "1D"
start_data = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_data = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat()

In [12]:
alpaca_df = alpaca.get_barset(
    ticker_alpaca, 
    timeframe, 
    start=start_data, 
    end=end_data
).df
alpaca_df.tail()

Empty DataFrame
Columns: [(BTC, open), (BTC, high), (BTC, low), (BTC, close), (BTC, volume)]
Index: []

**Conclusion: Not Selected, as No Avaliable Data from Alpaca.**

## 2) Testing & Importing Data from Quandl:

In [16]:
import quandl
load_dotenv(dotenv_path = Path("C:/Users/Leon/API_keys/.env"))
api_key_quandl = os.getenv("QUANDL_API_KEY")
type(api_key_quandl)

str

In [18]:
quandl_df = quandl.get("BCHAIN/MKPRU", start_date='2017-01-01', end_date='2020-12-31', collaps='daily', api_key= api_key_quandl)
quandl_df.tail()

,Value
Date,
2020-12-27,24693.58
2020-12-28,26443.21
2020-12-29,26246.58
2020-12-30,27036.69
2020-12-31,27376.37


**Conclusion: Not Selected, as Missing Volume Data from Quandl.**

## 3) Testing & Importing Data from FXCM:

Install fxcmpy:

In [23]:
# pip install fxcmpy
# pip install fxcmpy --upgrade
# pip install python-socketio==4.4.0

In [24]:
import fxcmpy
import socketio

In [25]:
con = fxcmpy.fxcmpy(access_token=str(os.getenv("FXCM")), log_level='error', server='demo', log_file='log.txt')

In [26]:
start_data = datetime(2017,1,1)
end_data = datetime(2020,12,31)
timeframe = "D1"

fxcm_df = con.get_candles('BTC/USD', period=timeframe, start=start_data, stop=end_data)
fxcm_df.tail()

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2020-12-24 22:00:00,23288.0,23333.0,23457.0,22616.8,23327.0,23379.0,23506.0,22666.0,392172
2020-12-28 22:00:00,26332.0,26573.6,27458.0,26073.1,26376.1,26618.0,27490.5,26111.0,498917
2020-12-29 22:00:00,26573.6,26881.0,27166.0,25822.0,26617.0,26933.0,27219.0,25872.8,417727
2020-12-30 22:00:00,26881.0,28887.0,29001.0,26881.0,26933.0,28941.0,29044.0,26927.0,454514
2020-12-31 22:00:00,28887.0,28866.3,29282.0,27945.2,28942.0,28943.7,29345.0,27975.0,405988


In [27]:
# Disconnect
con.close()

**Conclusion: Not Selected, due to: <br>1) "tick qty" is not volume but just an approximate for volume, and   <br> 2) need to convert the spread of bid/ask close to mid-close price.**

## 4) Testing & Importing Data from Alpha Vantage:

In [33]:
import requests
import json

In [38]:
demo = "8JM6BG1YFNGOXRGS"
url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_Daily&symbol=BTC&market=CNY&apikey={demo}"
response_data = requests.get(url)   
data = response_data.json()

In [39]:
data_json = pd.DataFrame.from_dict(data["Time Series (Digital Currency Daily)"]).T
data_json.tail()

,1a. open (CNY),1b. open (USD),2a. high (CNY),2b. high (USD),3a. low (CNY),3b. low (USD),4a. close (CNY),4b. close (USD),5. volume,6. market cap (USD)
2018-04-16,53986.28480500,8355.07000000,54399.36850000,8419.00000000,50832.62050000,7867.00000000,52111.48058000,8064.92000000,36664.06971500,36664.06971500
2018-04-15,51717.84600000,8004.00000000,54467.47271000,8429.54000000,51685.66773000,7999.02000000,53985.83250000,8355.00000000,27946.72044400,27946.72044400
2018-04-14,50900.33702000,7877.48000000,52893.83900000,8186.00000000,50464.31500000,7810.00000000,51685.60311500,7999.01000000,31621.28635700,31621.28635700
2018-04-13,51194.39988500,7922.99000000,53200.04948500,8233.39000000,49960.31800000,7732.00000000,50899.88471500,7877.41000000,55044.52314800,55044.52314800
2018-04-12,44931.84947000,6953.78000000,51771.02414500,8012.23000000,43571.18680000,6743.20000000,51194.46450000,7923.00000000,64861.59598700,64861.59598700


**Conclusion: Not Selected, due to missing data for 2017.**

## 5) Testing & Importing Data from Investing:

In [29]:
csvpath = Path("Resources/S&P 500 Historical Data.csv")
csv_df = pd.read_csv(csvpath, index_col="Date", parse_dates=True, infer_datetime_format=True).sort_values("Date")
csv_df.tail()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2020-12-14,"3,647.49","3,675.27","3,697.61","3,645.84",-,-0.44%
2020-12-15,"3,694.62","3,666.41","3,695.29","3,659.62",-,1.29%
2020-12-16,"3,701.17","3,696.25","3,711.27","3,688.57",-,0.18%
2020-12-17,"3,722.48","3,713.65","3,725.12","3,710.87",-,0.58%
2020-12-18,"3,708.80","3,723.63","3,726.30","3,708.80",-,-0.37%


In [31]:
csvpath = Path("Resources/Bitcoin Historical Data - Investing.com AU.csv")
csv_df = pd.read_csv(csvpath, index_col="Date", parse_dates=True, infer_datetime_format=True).sort_values("Date")
csv_df.tail()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2020-12-14,"19,273.8","19,176.4","19,346.5","19,007.0",67.11K,0.51%
2020-12-15,"19,434.9","19,273.9","19,556.3","19,076.2",98.39K,0.84%
2020-12-16,"21,352.2","19,434.7","21,525.3","19,299.7",199.81K,9.87%
2020-12-17,"22,825.4","21,352.3","23,738.0","21,243.1",310.74K,6.90%
2020-12-18,"22,575.7","22,823.3","23,272.8","22,420.2",195.96K,-1.09%


**Conclusion: Not Selected, due to <br> 1) No Volume Data from Investing. <br>2)  Extra cleaning need for Vol.**

### Other Data: M2 US Money Supply:

In [44]:
csvpath = Path("Resources/M2.csv")
m2_df = pd.read_csv(csvpath, index_col="DATE", parse_dates=True, infer_datetime_format=True).sort_values("DATE")
m2_df.rename(columns={"M2" : "M2(billions)"}, inplace=True)

# Checking Data Quality:
print(m2_df.dtypes)
print("\n")
print(m2_df.shape)
print("\n")
print(m2_df[m2_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(m2_df.isnull().sum())
print("\n")
print(m2_df.head())

M2(billions)    float64
dtype: object


(206, 1)


Empty DataFrame
Columns: [M2(billions)]
Index: []


M2(billions)    0
dtype: int64


            M2(billions)
DATE                    
2017-01-02       13214.2
2017-01-09       13257.3
2017-01-16       13274.1
2017-01-23       13292.6
2017-01-30       13289.6


**Comment : Investing provides other useful data which might be used for the project, such as M2 money supply of USD.
<br>But, there is a large amount of missing daily data.**

## 6) Testing & Importing Data from Yahoo Finance:


<span style=color:red> Install yfinance:

In [4]:
# !pip install yfinance
import yfinance as yf

### Bitcoin USD (BTC-USD):

In [6]:
yf_bitcoin_df = yf.download("BTC-USD", start="2017-01-01", end="2020-12-31")

# Checking Data Quality:
print(yf_bitcoin_df.dtypes)
print("\n")
print(yf_bitcoin_df.shape)
print("\n")
print(yf_bitcoin_df[yf_bitcoin_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(yf_bitcoin_df.isnull().sum())
print("\n")
print(yf_bitcoin_df.tail())

[*********************100%***********************]  1 of 1 downloaded
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


(1461, 6)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


                Open      High       Low     Close  Adj Close       Volume
Date                                                                      
2020-12-26  24677.02  26718.07  24522.69  26437.04   26437.04  48332647295
2020-12-27  26439.37  28288.84  25922.77  26272.29   26272.29  66479895605
2020-12-28  26280.82  27389.11  26207.64  27084.81   27084.81  49056742893
2020-12-29  27081.81  27370.72  25987.30  27362.44   27362.44  45265946774
2020-12-30  27360.09  28937.74  27360.09  28840.95   28840.95  51287442704


### U.S. Dollar Index (USDX) Futures Contract (DX=F):

In [40]:
yf_dollar_df = yf.download("DX=F", start="2017-01-01", end="2020-12-31")

# Checking Data Quality:
print(yf_dollar_df.dtypes)
print("\n")
print(yf_dollar_df.shape)
print("\n")
print(yf_dollar_df[yf_dollar_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(yf_dollar_df.isnull().sum())
print("\n")
print(yf_dollar_df.tail())

[*********************100%***********************]  1 of 1 downloaded
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


(1001, 6)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


              Open    High     Low   Close  Adj Close  Volume
Date                                                         
2020-12-22  90.100  90.620  89.950  90.546     90.546   21871
2020-12-23  90.535  90.580  90.050  90.340     90.340   27553
2020-12-28  90.260  90.320  89.895  90.275     90.275   18455
2020-12-29  90.160  90.165  89.785  89.918     89.918   17249
2020-12-30  89.865  89.900  89.515  89.649     89.649   23431


### Gold Futures Contract (GC=F):

In [41]:
yf_gold_df = yf.download("GC=F", start="2017-01-01", end="2020-12-31")

# Checking Data Quality:
print(yf_gold_df.dtypes)
print("\n")
print(yf_gold_df.shape)
print("\n")
print(yf_gold_df[yf_gold_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(yf_gold_df.isnull().sum())
print("\n")
print(yf_gold_df.tail())

[*********************100%***********************]  1 of 1 downloaded
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


(1079, 6)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


              Open    High     Low   Close  Adj Close  Volume
Date                                                         
2020-12-22  1882.2  1882.2  1861.0  1866.6     1866.6     233
2020-12-23  1867.0  1878.8  1864.5  1874.7     1874.7      82
2020-12-28  1895.5  1895.8  1873.7  1877.2     1877.2      75
2020-12-29  1881.3  1881.3  1879.7  1879.7     1879.7     343
2020-12-30  1879.9  1895.8  1877.3  1891.0     1891.0     444


### Other Data: S&P 500 Index (^GSPC)：

In [42]:
yf_sp500_df = yf.download("^GSPC", start="2017-01-01", end="2020-12-31")

# Checking Data Quality:
print(yf_sp500_df.dtypes)
print("\n")
print(yf_sp500_df.shape)
print("\n")
print(yf_sp500_df[yf_sp500_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(yf_sp500_df.isnull().sum())
print("\n")
print(yf_sp500_df.tail())

[*********************100%***********************]  1 of 1 downloaded
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


(1005, 6)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


               Open     High      Low    Close  Adj Close      Volume
Date                                                                 
2020-12-22  3698.08  3698.26  3676.16  3687.26    3687.26  4023940000
2020-12-23  3693.42  3711.24  3689.28  3690.01    3690.01  3772630000
2020-12-24  3694.03  3703.82  3689.32  3703.06    3703.06  1885090000
2020-12-28  3723.03  3740.51  3723.03  3735.36    3735.36  3527460000
2020-12-29  3750.01  3756.12  3723.31  3727.04    3727.04  3387030000


**Conclusion: Yahoo Finance is Selected, as it provides highest data quality comparing to other sources.**

## Rename Columns & Copy Original DataFrames with Selected Columns:

In [45]:
# Rename Columns:
selected_dollar_df = yf_dollar_df.copy()
selected_dollar_df.columns = [('dollar_'+ column) for column in selected_dollar_df.columns]
# Selected Columns:
selected_dollar_df = selected_dollar_df.iloc[:,[-2,-1]]
selected_dollar_df.tail(3)

,dollar_Adj Close,dollar_Volume
Date,,
2020-12-28,90.275,18455
2020-12-29,89.918,17249
2020-12-30,89.649,23431


In [46]:
# Rename Columns:
selected_gold_df = yf_gold_df.copy()
selected_gold_df.columns = [('gold_'+ column) for column in selected_gold_df.columns]
# Selected Columns:
selected_gold_df = selected_gold_df.iloc[:,[-2,-1]]
selected_gold_df.tail(3)

,gold_Adj Close,gold_Volume
Date,,
2020-12-28,1877.2,75
2020-12-29,1879.7,343
2020-12-30,1891.0,444


In [50]:
# Rename Columns:
selected_bitcoin_df = yf_bitcoin_df.copy()
selected_bitcoin_df.columns = [('bitcoin_'+ column) for column in selected_bitcoin_df.columns]
# Selected Columns:
selected_bitcoin_df = selected_bitcoin_df.iloc[:,[-2,-1]]
selected_bitcoin_df.tail(3)

,bitcoin_Adj Close,bitcoin_Volume
Date,,
2020-12-28,27084.81,49056742893
2020-12-29,27362.44,45265946774
2020-12-30,28840.95,51287442704


In [48]:
# Rename Columns:
selected_sp500_df = yf_sp500_df.copy()
selected_sp500_df.columns = [('sp500_'+ column) for column in selected_sp500_df.columns]
# Selected Columns:
selected_sp500_df = selected_sp500_df.iloc[:,[-2,-1]]
selected_sp500_df.tail(3)

,sp500_Adj Close,sp500_Volume
Date,,
2020-12-24,3703.06,1885090000
2020-12-28,3735.36,3527460000
2020-12-29,3727.04,3387030000


## Concatenating Selected DataFrames for Further  Analysis:

In [51]:
joined_df = pd.concat([selected_dollar_df, selected_gold_df, selected_bitcoin_df, selected_sp500_df], axis = 'columns', join='inner', sort=True)

# Checking Data Quality:
print(joined_df.dtypes)
print("\n")
print(joined_df.shape)
print("\n")
print(joined_df[joined_df.duplicated(keep=False)]) #False: Mark all duplicates as True
print("\n")
print(joined_df.isnull().sum())
print("\n")
print(joined_df.tail())

dollar_Adj Close     float64
dollar_Volume          int64
gold_Adj Close       float64
gold_Volume            int64
bitcoin_Adj Close    float64
bitcoin_Volume         int64
sp500_Adj Close      float64
sp500_Volume           int64
dtype: object


(994, 8)


Empty DataFrame
Columns: [dollar_Adj Close, dollar_Volume, gold_Adj Close, gold_Volume, bitcoin_Adj Close, bitcoin_Volume, sp500_Adj Close, sp500_Volume]
Index: []


dollar_Adj Close     0
dollar_Volume        0
gold_Adj Close       0
gold_Volume          0
bitcoin_Adj Close    0
bitcoin_Volume       0
sp500_Adj Close      0
sp500_Volume         0
dtype: int64


            dollar_Adj Close  dollar_Volume  gold_Adj Close  gold_Volume  \
Date                                                                       
2020-12-21            89.949          39253          1879.2          136   
2020-12-22            90.546          21871          1866.6          233   
2020-12-23            90.340          27553          1874.7           82